In [1]:
#mount google drive

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

In [3]:
#change directory
os.chdir('/content/drive/MyDrive/capstone')

In [4]:
import pandas as pd

In [5]:
#read in dataset

df = pd.read_csv('Copy of Food Health Claims- KG - 13. Authorised.csv')

In [6]:
df

,Status,Claim,Claim Type,Food,NEW Food Type,Food Type,Food Matrix OLD,Regulation specifications,NEW Food Matrix,Food Ontology Term,...,Supporting Evidence Text 5.1,Supporting Evidence Reference 5,Supporting Evidence Text 6,Supporting Evidence Reference 6,Supporting Evidence Text 7,Supporting Evidence Reference 7,Supporting Evidence Text 8,Supporting Evidence Reference 8,Supporting Evidence Text 9,Supporting Evidence Reference 9
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{matrix: ""food"", contains:{unit:[""g/100"", ""g/1...",NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Finished,ALA contributes to the maintenance of normal b...,Art. 13(1),Alpha-linolenic acid (ALA),Nutrient,Macronutrient,May be used only for food which is at least a ...,A claim that a food is a source of omega-3 fat...,Matrix: food (general)\nMust contain: ALA\n- U...,https://foodb.ca/compounds/FDB012462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Finished,Activated charcoal contributes to reducing exe...,Art. 13(1),Activated charcoal,Compound,BioactiveCompound,NaN,NaN,Matrix: food supplement\nMust contain: activat...,https://foodb.ca/compounds/FDB008898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Finished,Barley grain fibre contributes to an increase ...,Art. 13(1),Barley grain fibre,Nutrient,Macronutrient,May be used only for food which is high in tha...,"A claim that a food is high in fibre, and any ...",Matrix: food (general)\nMust contain: Barley g...,https://foodb.ca/foods/FOOD00088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Finished,Beta-glucans contribute to the maintenance of ...,Art. 13(1),Beta-glucans,Nutrient,Macronutrient,May be used only for food which contains at le...,NaN,Matrix: food (general)\nMust contain: beta-glu...,https://foodb.ca/compounds/FDB005762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
232,Finished,Consumption of foods/drinks containing [name o...,Art. 13(5),Non-digestible carbohydrates,Nutrient,Macronutrient,Sugars should be replaced in foods or drinks b...,A claim stating that the content in one or mor...,Unit: %\nvalue: -30\nprecondition: compared to...,https://foodb.ca/nutrients/FDBN00003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
233,Finished,Consumption of foods/drinks containing [name o...,Art. 13(5),"""Non-fermentable"" carbohydrates",Nutrient,Macronutrient,Fermentable carbohydrates should be replaced i...,NaN,Fermentable carbohydrates should be replaced i...,https://foodb.ca/nutrients/FDBN00003,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,Finished,Daily creatine consumption can enhance the eff...,Art. 13(5),Creatine,Nutrient,Macronutrient,"Foods targeting adults over the age of 55, who...",NaN,Matrix: foods targeted at people over 55.,https://foodb.ca/compounds/FDB005403,...,"The Panel notes that, in this opinion, the stu...",Chrusch et al. (2001): 10.1097/00005768-200112...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
235,Finished,Lactitol contributes to normal bowel function ...,Art. 13(5),Lactitol,Nutrient,Macronutrient,Food supplements which contain 10g of lactitol...,NaN,Unit: g/supplement\nValue: 10\nMatrix: supplement,https://foodb.ca/compounds/FDB020779,...,The studies submitted by the applicant in supp...,Patil et al. (1987a): 10.1079/bjn19870025; Pat...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
synonym_food = dict()
synonym_phenotype = dict()

In [8]:

def create_example(row):
  claim = str(row['Claim']).lower()
  food = str(row['Food']).lower()
  pheno = str(row['Phenotype']).lower()

#print each claim, so that if no match is found it can be easily added manually to the dictionary

  print (claim,' : ',  food,' : ',pheno )
  if food in claim:

#if food term is already one-to-one in the claim text, create the span and continue

    si_food = claim.index(food)
    fi_food = si_food+ len(food)
  else:

#if food term is not in the claim text, check the synonym dictionary. If the term does not exist, it needs to be manually added

    for s in synonym_food[food]:
      if s in claim:
        si_food = claim.index(s)
        fi_food = si_food+ len(s)
        continue

#if phenotype term is already one-to-one in the claim text, create the span and continue

  if pheno in claim:  
    si_pheno = claim.index(pheno)
    fi_pheno = si_pheno +len(pheno)
  else:

#if phenotype term is not in the claim text, check the synonym dictionary. If the term does not exist, it needs to be manually added

    for s in synonym_phenotype[pheno]:
      if s in claim:
         si_pheno = claim.index(s)
         fi_pheno = si_pheno+ len(s)
         continue
  entity_mentions = [(si_food,fi_food,'Food'), (si_pheno,fi_pheno, 'Phenotype')]
  return (claim, {'entities' : entity_mentions})

In [9]:
#test function for the above function
def test_create_example_1():
  row = dict()
  row['Claim']= 'Lactitol contributes to normal bowel function by increasing stool frequency'
  row['Food'] = 'Lactitol'
  row['Phenotype'] = 'Normal bowel function'
  result = create_example(row)
  print (result)
  assert (result[1] == (0,8,'Food'))
  assert (result[2] == (24, 45, 'Phenotype'))

In [10]:
#test_create_example_1()

In [11]:
def test_create_example_2():
  row = dict()
  row['Claim']= 'ALA contributes to the maintenance of normal blood cholesterol levels'
  row['Food'] = 'Alpha-linolenic acid (ALA)'
  row['Phenotype'] = 'Normal blood cholesterol'
  result = create_example(row)
  print (result)
  assert (result[1] == (0,3,'Food'))
  assert (result[2] == (38, 62, 'Phenotype'))

In [12]:
#test_create_example_2()

In [13]:
#drop missing values
df.dropna(subset=['Status'], inplace=True)

In [14]:
df.head()

,Status,Claim,Claim Type,Food,NEW Food Type,Food Type,Food Matrix OLD,Regulation specifications,NEW Food Matrix,Food Ontology Term,...,Supporting Evidence Text 5.1,Supporting Evidence Reference 5,Supporting Evidence Text 6,Supporting Evidence Reference 6,Supporting Evidence Text 7,Supporting Evidence Reference 7,Supporting Evidence Text 8,Supporting Evidence Reference 8,Supporting Evidence Text 9,Supporting Evidence Reference 9
1,Finished,ALA contributes to the maintenance of normal b...,Art. 13(1),Alpha-linolenic acid (ALA),Nutrient,Macronutrient,May be used only for food which is at least a ...,A claim that a food is a source of omega-3 fat...,Matrix: food (general)\nMust contain: ALA\n- U...,https://foodb.ca/compounds/FDB012462,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Finished,Activated charcoal contributes to reducing exe...,Art. 13(1),Activated charcoal,Compound,BioactiveCompound,NaN,NaN,Matrix: food supplement\nMust contain: activat...,https://foodb.ca/compounds/FDB008898,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Finished,Barley grain fibre contributes to an increase ...,Art. 13(1),Barley grain fibre,Nutrient,Macronutrient,May be used only for food which is high in tha...,"A claim that a food is high in fibre, and any ...",Matrix: food (general)\nMust contain: Barley g...,https://foodb.ca/foods/FOOD00088,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Finished,Beta-glucans contribute to the maintenance of ...,Art. 13(1),Beta-glucans,Nutrient,Macronutrient,May be used only for food which contains at le...,NaN,Matrix: food (general)\nMust contain: beta-glu...,https://foodb.ca/compounds/FDB005762,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Finished,Betaine contributes to normal homocysteine met...,Art. 13(1),Betaine,Compound,BioactiveCompound,May be used only for food which contains at le...,NaN,Matrix: food (general)\nMust contain: betaine\...,https://foodb.ca/compounds/FDB009020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
#add to the synonym dictionary for foods for the authorised datset
synonym_food['Alpha-linolenic acid (ALA)'.lower()] = ['ALA'.lower()]
synonym_food['hydroxypropyl methylcellulose (hpmc)'.lower()] = ['hydroxypropyl methylcellulose']
synonym_food['arabinoxylan produced from wheat endosperm'.lower()] = ['arabinoxylan']
synonym_food['beta-glucans from oats and barley']  = ['beta-glucans from oats or barley']
synonym_food['sugar replacers, i.e. intense sweeteners; xylitol, sorbitol, mannitol, maltitol, lactitol, isomalt, erythritol, sucralose and polydextrose; d-tatatose and isomaltulose'.lower()] = ['[name of sugar replacer]']
synonym_food['docosahexaenoic acid and eicosapentaenoic acid (dha/epa)'] = ['dha and epa']
synonym_food['docosahexaenoic acid (dha)'] = ['dha']
synonym_food['dried plums of \'prune\' cultivars (prunus domestica)'] = ['dried plums/prunes']
synonym_food['eicosapentaenoic acid and docosahexaenoic acid (epa/dha)'] = ['epa and dha']
synonym_food['glucomannan (konjac mannan)'] = ['glucomannan']
synonym_food['live yoghurt cultures'] = ['live cultures in yoghurt']
synonym_food['monascus purpureous (red yeast rice)'] = ['red yeast rice']
synonym_food['plant sterols and plant stanols'] = ['plant sterols/stanols']
synonym_food['foods with a low or reduced content of saturated fatty acids'] = ['reducing consumption of saturated fat']
synonym_food['foods with a low or reduced content of sodium'] = ['reducing consumption of sodium']
synonym_food['monounsaturated and/or polyunsaturated fatty acids'] = ['unsaturated fats']
synonym_food['riboflavin (vitamin b2)'] = ['riboflavin']
synonym_food['meal replacement for weight control'] =  ['meal replacement']
synonym_food['"non-fermentable" carbohydrates'] = ['[name of all used non-fermentable carbohydrates]']


In [16]:
#create to the synonym dictionary for phenotypes for the authorised dataset
synonym_phenotype['excessive flatulence'] = ['exessive flatulence']
synonym_phenotype['normal blood homocysteine'] = ['normal homocysteine metabolism']
synonym_phenotype['endurance performance'] = ['endurance performing']
synonym_phenotype['enhancement of water absorption during physical exercise'] = ['enhance the absorption of water during physical exercise']
synonym_phenotype['normal digestion by gastric acid'] = ['normal digestion']
synonym_phenotype['increased blood glucose'] = ['blood glucose rise']
synonym_phenotype['normal tooth mineralisation'] = ['maintenance of tooth mineralisation', 'maintenance of tooth mineralization']
synonym_phenotype['pigment metabolic process involved in pigment accumulation'] = ['normal hair pigmentation']
synonym_phenotype['normal immune system function'] = ['normal function of the immune system']
synonym_phenotype['increased physical performance'] = ['increases the physical performance']
synonym_phenotype['blood pressure normal'] = ['normal blood pressure']
synonym_phenotype['blood triglycerides normal'] = ['normal blood triglyceride levels']
synonym_phenotype['normal cardiac function '] = ['normal function of the heart']
synonym_phenotype['normal homocysteine level'] = ['normal homocysteine metabolism']
synonym_phenotype['cell division '] = ['cell division', 'cell specialisation']
synonym_phenotype['normal red blood cell formation and normal haemoglobin formation'] = ['normal formation of red blood cells and haemoglobin']
synonym_phenotype['decreased tiredness and fatigue'] = ['reduction of tiredness and fatigue']
synonym_phenotype['lactose intolerance'] = ['difficulty digesting lactose']
synonym_phenotype['intestinal transit time decreased'] = ['acceleration of intestinal transit']
synonym_phenotype['increased iron absorption'] = ['improvement of iron absorption', 'increases iron absorption']
synonym_phenotype['decreased feelings of jet lag'] = ['allevation of subjective feelings of jet lag']
synonym_phenotype['reduction of sleep onset latency'] = ['reduction of time taken to fall asleep']
synonym_phenotype['protection of cells from oxidative stress'] = ['protection of blood lipids from oxidative stress']
synonym_phenotype['normal synthesis and metabolism of \n- steroid hormones\n- vitamin d\n- some neurotransmitters'] = ['normal synthesis and metabolism of steroid hormones, vitamin d and some neurotransmitters']
synonym_phenotype['increased muscle mass'] = ['growth in muscle mass']
synonym_phenotype['normal iron metabolism'] = ['normal metabolism of iron']
synonym_phenotype['reduced weight'] = ['weight loss']
synonym_phenotype['dental plaque'] = ['plaque acids']
synonym_phenotype['homocysteine level'] = ['homocysteine metabolism']
synonym_phenotype['normal red blood cell formation '] = ['normal red blood cell formation']
synonym_phenotype['normal protein metabolism and normal glycogen metabolism'] = ['norrmal protein and glycogen metabolism']
synonym_phenotype['hormone activity'] = ['hormonal activity']
synonym_phenotype['maintenance of immune system after exercise'] = ['maintain the normal function of the immune system during and after intense physical exercise']
synonym_phenotype['regeneration of reduced form of vitamin e'] = ['regeneration of the reduced form of vitamin e']
synonym_phenotype['normal absorption of calcium and normal absorption of phosphorus']  = ['normal absorption/utilisation of calcium and phosphorus']
synonym_phenotype['increased vessel elasticity'] = ['improvement of the elasticity of blood vessels']
synonym_phenotype['body temperature normal'] = ['normal regulation of the body\'s temperature']
synonym_phenotype['intestinal transit time increased'] = ['acceleration of intestinal transit']
synonym_phenotype['normal fatty acid metabolism'] = ['normal metabolism of fatty acids']
synonym_phenotype['normal vitamin a metabolism'] = ['normal metabolism of vitamin a']
synonym_phenotype['enhanced muscle strength'] = ['enhance the effect of resistance training on muscle strength']


In [17]:
data_labeled = df.apply(create_example, axis=1)

ala contributes to the maintenance of normal blood cholesterol levels  :  alpha-linolenic acid (ala)  :  normal blood cholesterol
activated charcoal contributes to reducing exessive flatulence after eating  :  activated charcoal  :  excessive flatulence
barley grain fibre contributes to an increase in faecal bulk  :  barley grain fibre  :  increase in faecal bulk
beta-glucans contribute to the maintenance of normal blood cholesterol levels  :  beta-glucans  :  normal blood cholesterol
betaine contributes to normal homocysteine metabolism  :  betaine  :  normal blood homocysteine
biotin contributes to normal energy-yielding metabolism  :  biotin  :  normal energy-yielding metabolism
biotin contributes to normal functioning of the nervous system  :  biotin  :  normal functioning of the nervous system
biotin contributes to normal macronutrient metabolism  :  biotin  :  normal macronutrient metabolism
biotin contributes to normal psychological function  :  biotin  :  normal psychological f

In [18]:
len(data_labeled)

235

In [19]:
data_labeled_list = data_labeled.tolist()
data_labeled_list

[('ala contributes to the maintenance of normal blood cholesterol levels',
  {'entities': [(0, 3, 'Food'), (38, 62, 'Phenotype')]}),
 ('activated charcoal contributes to reducing exessive flatulence after eating',
  {'entities': [(0, 18, 'Food'), (43, 62, 'Phenotype')]}),
 ('barley grain fibre contributes to an increase in faecal bulk',
  {'entities': [(0, 18, 'Food'), (37, 60, 'Phenotype')]}),
 ('beta-glucans contribute to the maintenance of normal blood cholesterol levels',
  {'entities': [(0, 12, 'Food'), (46, 70, 'Phenotype')]}),
 ('betaine contributes to normal homocysteine metabolism',
  {'entities': [(0, 7, 'Food'), (23, 53, 'Phenotype')]}),
 ('biotin contributes to normal energy-yielding metabolism',
  {'entities': [(0, 6, 'Food'), (22, 55, 'Phenotype')]}),
 ('biotin contributes to normal functioning of the nervous system',
  {'entities': [(0, 6, 'Food'), (22, 62, 'Phenotype')]}),
 ('biotin contributes to normal macronutrient metabolism',
  {'entities': [(0, 6, 'Food'), (22, 53

In [20]:
# split into training and test
train_size = int(len(data_labeled)*9/10)
test_size = len(data_labeled) - train_size
test_size

24

In [33]:
#create a file that will save annotation in jsonl format for usage in prodigy
import json

f = open('authorised_annotated.jsonl', 'w')


In [34]:
#import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(data_labeled_list[:-test_size]): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    dicts = {}
    dicts['text'] = text
    spans = []

    for start, end, label in annot["entities"]: # add character indexes
        #print (start, end, label)
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
            spans.append({'text': text[start:end], 'start': start, 'end': end, 'label': label})

#save annotations to the previously created jsonl file
    dicts['spans'] = spans
    string = json.dumps(dicts)
    f.write(string+'\n')
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("train.spacy") # save the docbin object

db = DocBin()
for text, annot in tqdm(data_labeled_list[-test_size:]): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label, alignment_mode="strict")
        if span is None:
            print("Skipping entity")
            print (start, end, label)
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("valid.spacy") # save the docbin object

100%|██████████| 211/211 [00:00<00:00, 1496.84it/s]


Skipping entity
41 52 Phenotype
Skipping entity
37 61 Phenotype
Skipping entity
22 45 Phenotype
Skipping entity
44 55 Phenotype
Skipping entity
44 55 Phenotype
Skipping entity
29 66 Phenotype
Skipping entity
26 58 Phenotype
Skipping entity
45 56 Phenotype
Skipping entity
42 53 Phenotype
Skipping entity
100 122 Phenotype
Skipping entity
63 79 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
0 10 Food
Skipping entity
44 55 Phenotype
Skipping entity
44 55 Phenotype
Skipping entity
40 78 Phenotype


100%|██████████| 24/24 [00:00<00:00, 968.70it/s]


Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
39 50 Phenotype
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food
Skipping entity
0 5 Food


In [35]:
f.close()